In [ ]:
import numpy as np
from scipy.special import i0,i1 #Modified Bessel functions
from scipy.integrate import quad
# from decimal import Decimal, getcontext
# getcontext().prec = 5

In [ ]:
# Using scipy.integrate.quad to calculate Goldstein's J-function J(a,b)
def gold(a, b):
    def f(x,b):
        m1 = i0(2*np.sqrt(b*x))
        return np.exp(-x)*m1

    # Perform the integration
    result, _ = quad(f, 0, a, args=(b,))
    return 1 - np.exp(-b) * result

In [ ]:
V = 20.0
D = 10.0
R = 5.0
Beta = 0.44
omega = 0.56
Mu1  = 0.0
Mu2  = 0.0

Zl = 50.0

In [ ]:
P = V * Zl / D
betr = Beta*R
omegamu2 = omega+Mu2

a = omega * omega / omegamu2 / betr
b = omegamu2 / (R - betr)

cx = omega / omegamu2

da = omega * Mu2 / omegamu2

In [ ]:
def cc0(tau):
    dg = np.exp(-Mu1 * tau / betr)
    g1 = np.sqrt(betr * P / (4.0 * np.pi * tau))
    g2 = np.exp(P * (betr * ZZ - tau) * (tau - betr * ZZ) / (4.0 * betr * tau))

    return ZZ / tau * dg * g1 * g2

In [ ]:
def c1bj(tau,TTT):
    
    g1 = cc0(tau) 
    if g1 < 1e-7 :
        return 0.0
    at = a * tau
    bt = b * (TTT - tau)
    g2 = np.exp(-Mu2 * at / omega)
    g3 = gold(at,bt)

    return g1 * g2 *g3

In [ ]:
def c2bj(tau,TTT):
    g1 = cc0(tau) 
    if g1 < 1e-7 :
        return 0.0
    
    at = a * tau
    bt = b * (TTT - tau)
    
    g2 = np.exp(-Mu2 * at / omega)

    g3 = 1.0 - gold(bt,at)

    return cx * g1 * g2 * g3

In [ ]:
nt = 101 
dt = 0.20

# pulses = [{'conc': 1.0, 'time': 0.0}, {'conc': 0.0, 'time': 5.0}]
npulse = 2
cpulse = [1.0, 0.0]
tpulse = [0.0, 5.0]

tpulse = [i * V / Zl for i in tpulse]

In [ ]:
def bound():
    if TT < 1e-7 :
        return 0.0, 0.0
    
    TTT = TT
    a1, _ = quad(c1bj, 0, TTT, args=(TTT,))
    a2, _ = quad(c2bj, 0, TTT, args=(TTT,))
    C1 = cpulse[0] * a1
    C2 = cpulse[0] * a2
    
    for i in range(1,npulse) :
        TTT = TTT - tpulse[i]
        if TTT <= 0 :
            return C1, C2
        a1, _ = quad(c1bj, 0, TTT, args=(TTT,))
        a2, _ = quad(c2bj, 0, TTT, args=(TTT,))
        C1 = C1 + (cpulse[i] - cpulse[i - 1]) * a1
        C2 = C2 + (cpulse[i] - cpulse[i - 1]) * a2
    return C1, C2            

In [ ]:
ZZ = 1.0
for i in range(nt):
    TT = i * dt * V / Zl

    C1, C2 = bound()
    print(f"{TT:.2f} {ZZ:.2f} {C1:.2E} {C2:.2E}")

In [ ]:
for i in range(1,npulse) :
    print(cpulse[i],tpulse[i])
